In [2]:
# from datasets import load_dataset
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score
from torchvision import transforms
import os
import sys
from huggingface_hub import snapshot_download
from PIL import Image
from typing import Tuple, List
import os
import random
from concurrent.futures import ThreadPoolExecutor
from typing import List, Tuple
import os
import numpy as np
import random
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset


c:\Users\sathi\miniconda3\envs\dl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [5]:
import os
import random
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

# Define transformations
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class Birddataset(Dataset):
    def __init__(self, image_dir: str, allowed_classes: list, transform=None, dataset_type: str = 'train'):
        """
        Args:
            image_dir (str): Path to the directory containing class subdirectories with images.
            allowed_classes (list): List of allowed class names to include.
            transform: Transformations to apply to images.
            dataset_type (str): Either 'train' or 'test' to control which samples to load.
        """
        self.image_dir = image_dir
        self.allowed_classes = allowed_classes
        self.transform = transform
        self.dataset_type = dataset_type

        # Prepare dataset
        self.images = self.prepare_dataset(image_dir, allowed_classes)

    def get_class_samples(self, class_dir: str, class_name: str) -> list:
        """
        Get all image file paths for a particular class.

        Args:
            class_dir (str): Directory of the class.
            class_name (str): Name of the class.

        Returns:
            list: List of (image_path, class_name) tuples.
        """
        return [(os.path.join(class_dir, img_entry.name), class_name) 
                for img_entry in os.scandir(class_dir) if img_entry.is_file()]

    def prepare_dataset(self, image_dir: str, allowed_classes: list) -> list:
        """
        Prepares the dataset by getting paths of images and splitting them into train and test sets.

        Args:
            image_dir (str): Path to the root directory containing class folders.
            allowed_classes (list): List of allowed class names.

        Returns:
            list: List of (image_path, class_name) tuples.
        """
        samples = []

        # Scan directories and process files in parallel
        with ThreadPoolExecutor() as executor:
            futures = []
            for class_name in os.listdir(image_dir):
                class_path = os.path.join(image_dir, class_name)
                if os.path.isdir(class_path) and class_name in allowed_classes:
                    futures.append(executor.submit(self.get_class_samples, class_path, class_name))

            for future in futures:
                class_samples = future.result()
                random.seed(42)
                random.shuffle(class_samples)
                
                # Split samples based on dataset type (train or test)
                split_idx = int(0.8 * len(class_samples))  # 80-20 train-test split
                if self.dataset_type == 'train':
                    samples.extend(class_samples[:split_idx])
                else:
                    samples.extend(class_samples[split_idx:])

        # Shuffle final list for randomness
        random.seed(42)
        random.shuffle(samples)
        print(len(samples))
        return samples

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, index: int) -> tuple:
        """
        Retrieves an image and its corresponding class label.

        Args:
            index (int): Index of the sample to retrieve.

        Returns:
            tuple: Transformed image tensor and class index.
        """
        image_path, class_name = self.images[index]

        # Load the image
        image = Image.open(image_path).convert('RGB')
        
        # Apply transformation
        if self.transform:
            image = self.transform(image)

        # Get class index
        class_id = self.allowed_classes.index(class_name)

        return image, class_id

In [6]:
train_set = Birddataset("dataset",["budgie","canary","duckling","rubber duck"])

119


In [58]:
a, b = Birddataset.prepare_dataset(self, image_dir="dataset",allowed_classes=["budgie","canary","duckling","rubber duck"])

NameError: name 'self' is not defined

In [ ]:

#Dataset function called
class Birddataset(Dataset):
    def __init__(self, image_dir: str, allowed_classes: List, transform=None, dataset_type: str = None):
        # Initialize paths and transformation
        self.image_dir = image_dir
        self.allowed_classes = allowed_classes
        self.transform = transform
        self.dataset_type = dataset_type



    def get_class_samples(self, class_dir: str, class_name: str) -> List[Tuple[str, str]]:
        # Lazy file reading with os.scandir, which is faster and memory efficient
        return [(os.path.join(class_dir, img_entry.name), class_name) 
                for img_entry in os.scandir(class_dir) if img_entry.is_file()]

    def prepare_dataset(self, image_dir: str, allowed_classes: List[str], transform=None) -> Tuple[List[Tuple[str, str]], List[Tuple[str, str]]]:
        train_samples = []
        test_samples = []

        # Scan directories only once
        with ThreadPoolExecutor() as executor:
            futures = []
            for class_name in os.listdir(image_dir):
                class_path = os.path.join(image_dir, class_name)
                if os.path.isdir(class_path) and (class_name in allowed_classes or class_name == "unlabeled"):
                    # Use thread pool for parallel file processing
                    futures.append(executor.submit(self.get_class_samples, class_path, class_name))

            for future in futures:
                class_samples = future.result()

                # Handle 'unlabeled' case separately
                if class_samples[0][1] == "unlabeled":
                    train_samples.extend(class_samples)
                else:
                    # Split train and test samples
                    random.seed(42)
                    random.shuffle(class_samples)
                    train_samples.extend(class_samples[:-3])
                    test_samples.extend(class_samples[-3:])

        # Final shuffling of train and test samples
        random.seed(42)
        random.shuffle(train_samples)
        random.shuffle(test_samples)

    return train_samples, test_samples
    def __len__(self) -> int:
        """
        Returns:
            int: The total number of image-mask pairs in the designated dataset split.
        """
        # Return the length of the dataset (number of images)
        return len(self.images)


    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Args:
            index (int): Index of the image-mask pair to retrieve.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: A tuple containing the image and its corresponding one-hot encoded mask.
                - image (torch.Tensor): Transformed image tensor.
                - onehot_mask (torch.Tensor): One-hot encoded mask tensor for segmentation.
        """
        # # Load the image and mask
        # image_path = os.path.join(self.image_dir,self.images[index][1],self.images[index][0])



        # # Load image and mask as grayscale
        # image = Image.open(image_path)
        # if self.transform:
        #     transformed = self.transform(image)
        # else:
        #     transformed = transform_test(image)

        # class_id = self.allowed_classes.index(self.images[index][1])

        # return transformed, class_id

use this one to prepare torch dataset object and use this transform transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
]) prepare in such a way that it is memory and time efficient
